# Pipeline ETL con Dask - Versión Limpia
=====================================

**Autor**: Ing. Daniel Varela Perez  
**Email**: bedaniele0@gmail.com  
**Teléfono**: +52 55 4189 3428  
**Fecha**: 24 de Septiembre, 2025  

**Objetivo**: Pipeline ETL limpio y funcional para procesamiento distribuido de datos de fraude

In [1]:
"""
Pipeline ETL con Dask - Versión Limpia
=====================================
Autor: Ing. Daniel Varela Perez
"""
# Imports principales
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import psutil
import os
import time
import warnings
from datetime import datetime
from pathlib import Path
import pickle
import json

# ML imports
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE

# Configuraciones
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 20)

# Crear directorios
Path('../data/processed').mkdir(parents=True, exist_ok=True)

# Función de monitoreo
def monitor_memory_usage(stage=""):
    """Monitorea memoria del proceso"""
    try:
        process = psutil.Process(os.getpid())
        mem_mb = process.memory_info().rss / 1024 / 1024
        print(f"📊 Memoria - {stage}: {mem_mb:.2f} MB")
    except Exception as e:
        print(f"⚠️ Error memoria: {e}")

print("🚀 PIPELINE ETL LIMPIO - DETECCIÓN DE FRAUDE")
print(f"📊 Desarrollado por: Ing. Daniel Varela Perez")
print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("✅ Setup completado")
monitor_memory_usage("Setup inicial")

🚀 PIPELINE ETL LIMPIO - DETECCIÓN DE FRAUDE
📊 Desarrollado por: Ing. Daniel Varela Perez
📅 Fecha: 2025-09-24 15:09
✅ Setup completado
📊 Memoria - Setup inicial: 227.95 MB


## 1. Carga de Datos con Dask

In [3]:
# Cargar dataset con Dask
print("📂 CARGANDO DATOS...")

data_path = '../data/raw/creditcard.csv'
ddf = dd.read_csv(data_path,
                  blocksize='50MB',
                  dtype={'Time': 'float64'},
                  assume_missing=True)

print(f"✅ Datos cargados")
print(f"📏 Dimensiones: {ddf.npartitions} particiones")
print(f"📋 Columnas: {len(ddf.columns)}")

# Info básica
with ProgressBar():
    total_rows = len(ddf)
    fraud_count = ddf['Class'].sum().compute()
    
print(f"📊 Total transacciones: {total_rows:,}")
print(f"🚨 Total fraudes: {fraud_count:,} ({fraud_count/total_rows*100:.4f}%)")

monitor_memory_usage("Datos cargados")

📂 CARGANDO DATOS...
✅ Datos cargados
📏 Dimensiones: 3 particiones
📋 Columnas: 31
[########################################] | 100% Completed | 1.27 ss
[########################################] | 100% Completed | 1.17 ss
📊 Total transacciones: 284,807
🚨 Total fraudes: 492.0 (0.1727%)
📊 Memoria - Datos cargados: 536.68 MB


## 2. Feature Engineering Básico

In [4]:
# Feature engineering simple y efectivo
print("⚡ CREANDO FEATURES...")

def create_features(partition):
    """Crea features básicos por partición"""
    
    # Features temporales
    partition['hour_from_start'] = partition['Time'] / 3600
    partition['day_from_start'] = partition['Time'] / 86400
    
    # Features de monto
    partition['amount_log'] = np.log1p(partition['Amount'])
    partition['is_zero_amount'] = (partition['Amount'] == 0).astype(int)
    partition['is_high_amount'] = (partition['Amount'] > 1000).astype(int)
    
    # Z-score local
    amount_mean = partition['Amount'].mean()
    amount_std = partition['Amount'].std()
    if amount_std > 0:
        partition['amount_zscore'] = (partition['Amount'] - amount_mean) / amount_std
    else:
        partition['amount_zscore'] = 0
    
    # Interacciones V1-V4 (principales)
    partition['V1_x_V2'] = partition['V1'] * partition['V2']
    partition['V3_x_V4'] = partition['V3'] * partition['V4']
    
    # Agregaciones
    v_cols_main = ['V1', 'V2', 'V3', 'V4', 'V5']
    partition['V_sum_main'] = partition[v_cols_main].sum(axis=1)
    partition['V_mean_main'] = partition[v_cols_main].mean(axis=1)
    
    return partition

# Aplicar feature engineering
with ProgressBar():
    ddf_features = ddf.map_partitions(create_features)

print(f"✅ Features creados")
print(f"📊 Total columnas: {len(ddf_features.columns)}")

# Nuevas features
new_features = [col for col in ddf_features.columns if col not in ddf.columns]
print(f"🔧 Nuevas features: {new_features}")

monitor_memory_usage("Features creados")

⚡ CREANDO FEATURES...
✅ Features creados
📊 Total columnas: 41
🔧 Nuevas features: ['hour_from_start', 'day_from_start', 'amount_log', 'is_zero_amount', 'is_high_amount', 'amount_zscore', 'V1_x_V2', 'V3_x_V4', 'V_sum_main', 'V_mean_main']
📊 Memoria - Features creados: 536.76 MB


## 3. Split Temporal

In [5]:
# Split temporal para evitar data leakage
print("📅 CREANDO SPLIT TEMPORAL...")

with ProgressBar():
    # Obtener rango temporal
    time_stats = ddf_features['Time'].describe().compute()
    min_time = time_stats['min']
    max_time = time_stats['max']
    time_range = max_time - min_time

# Puntos de corte (70% train, 15% val, 15% test)
train_cutoff = min_time + (time_range * 0.70)
val_cutoff = min_time + (time_range * 0.85)

print(f"📊 Rango temporal: {time_range/3600:.1f} horas")
print(f"📊 Train hasta: {train_cutoff/3600:.1f}h (70%)")
print(f"📊 Val: {train_cutoff/3600:.1f}h - {val_cutoff/3600:.1f}h (15%)")
print(f"📊 Test desde: {val_cutoff/3600:.1f}h (15%)")

# Crear splits
train_ddf = ddf_features[ddf_features['Time'] < train_cutoff]
val_ddf = ddf_features[(ddf_features['Time'] >= train_cutoff) & (ddf_features['Time'] < val_cutoff)]
test_ddf = ddf_features[ddf_features['Time'] >= val_cutoff]

# Verificar tamaños
with ProgressBar():
    train_size = len(train_ddf)
    val_size = len(val_ddf)
    test_size = len(test_ddf)
    
    train_fraud = train_ddf['Class'].sum().compute()
    val_fraud = val_ddf['Class'].sum().compute()
    test_fraud = test_ddf['Class'].sum().compute()

print(f"✅ SPLITS CREADOS:")
print(f"• Train: {train_size:,} filas, {train_fraud} fraudes ({train_fraud/train_size*100:.4f}%)")
print(f"• Val: {val_size:,} filas, {val_fraud} fraudes ({val_fraud/val_size*100:.4f}%)")
print(f"• Test: {test_size:,} filas, {test_fraud} fraudes ({test_fraud/test_size*100:.4f}%)")

monitor_memory_usage("Splits creados")

📅 CREANDO SPLIT TEMPORAL...
[########################################] | 100% Completed | 1.41 ss
📊 Rango temporal: 48.0 horas
📊 Train hasta: 33.6h (70%)
📊 Val: 33.6h - 40.8h (15%)
📊 Test desde: 40.8h (15%)
[########################################] | 100% Completed | 1.38 ss
[########################################] | 100% Completed | 1.46 ss
[########################################] | 100% Completed | 1.33 ss
[########################################] | 100% Completed | 1.34 ss
[########################################] | 100% Completed | 1.34 ss
[########################################] | 100% Completed | 1.33 ss
✅ SPLITS CREADOS:
• Train: 172,090 filas, 360.0 fraudes (0.2092%)
• Val: 59,587 filas, 61.0 fraudes (0.1024%)
• Test: 53,130 filas, 71.0 fraudes (0.1336%)
📊 Memoria - Splits creados: 463.61 MB


## 4. Preparación para ML

In [6]:
# Preparar features para ML
print("🎯 PREPARANDO PARA ML...")

# Columnas a excluir
exclude_cols = ['Time', 'Class']
feature_columns = [col for col in ddf_features.columns if col not in exclude_cols]
target_column = 'Class'

print(f"📋 Features para ML: {len(feature_columns)}")
print(f"🎯 Target: {target_column}")

# Crear scaler con muestra del train
print("📏 Creando scaler...")
with ProgressBar():
    train_sample = train_ddf[feature_columns].sample(frac=0.1).compute()

scaler = RobustScaler()
scaler.fit(train_sample)

print(f"✅ Scaler ajustado con muestra de {len(train_sample):,} registros")

# Función para normalizar particiones
def normalize_partition(partition, feature_cols):
    """Normaliza features en una partición"""
    partition_norm = partition.copy()
    if len(partition_norm) > 0:
        normalized_features = scaler.transform(partition_norm[feature_cols])
        partition_norm[feature_cols] = normalized_features
    return partition_norm

monitor_memory_usage("Scaler preparado")

🎯 PREPARANDO PARA ML...
📋 Features para ML: 39
🎯 Target: Class
📏 Creando scaler...
[########################################] | 100% Completed | 1.45 ss
✅ Scaler ajustado con muestra de 17,209 registros
📊 Memoria - Scaler preparado: 427.25 MB


## 5. Demo de Balanceado

In [7]:
# Demo de balanceado con SMOTE en muestra pequeña
print("⚖️ DEMO BALANCEADO...")

# Obtener muestra para demo
with ProgressBar():
    demo_sample = train_ddf.sample(frac=0.02).compute()  # 2% del train

print(f"📊 Muestra para demo: {len(demo_sample):,} registros")
print(f"🚨 Fraudes en muestra: {demo_sample['Class'].sum()}")

if demo_sample['Class'].sum() >= 5:  # Solo si hay suficientes fraudes
    
    X_demo = demo_sample[feature_columns]
    y_demo = demo_sample[target_column]
    
    # Estadísticas antes
    fraud_before = y_demo.sum()
    normal_before = len(y_demo) - fraud_before
    
    print(f"\n📊 ANTES DEL BALANCEADO:")
    print(f"• Normal: {normal_before:,} ({normal_before/len(y_demo)*100:.2f}%)")
    print(f"• Fraude: {fraud_before:,} ({fraud_before/len(y_demo)*100:.2f}%)")
    print(f"• Ratio: {normal_before/fraud_before:.1f}:1")
    
    try:
        # Aplicar SMOTE
        smote = SMOTE(random_state=42, k_neighbors=3)
        X_balanced, y_balanced = smote.fit_resample(X_demo, y_demo)
        
        # Estadísticas después
        fraud_after = y_balanced.sum()
        normal_after = len(y_balanced) - fraud_after
        
        print(f"\n📊 DESPUÉS DEL BALANCEADO:")
        print(f"• Normal: {normal_after:,} ({normal_after/len(y_balanced)*100:.2f}%)")
        print(f"• Fraude: {fraud_after:,} ({fraud_after/len(y_balanced)*100:.2f}%)")
        print(f"• Ratio: {normal_after/fraud_after:.1f}:1")
        print(f"• Tamaño final: {len(y_balanced):,} ({len(y_balanced)/len(y_demo):.1f}x)")
        
        print("✅ SMOTE demo completado")
        
    except Exception as e:
        print(f"⚠️ Error en SMOTE: {e}")
        print("Continuando sin balanceado...")
        
else:
    print("⚠️ Insuficientes fraudes para demo de balanceado")

monitor_memory_usage("Demo balanceado")

⚖️ DEMO BALANCEADO...
[########################################] | 100% Completed | 2.84 ss
📊 Muestra para demo: 3,442 registros
🚨 Fraudes en muestra: 4.0
⚠️ Insuficientes fraudes para demo de balanceado
📊 Memoria - Demo balanceado: 498.43 MB


## 6. Guardado de Datasets

In [8]:
# Guardar datasets procesados
print("💾 GUARDANDO DATASETS...")

# Normalizar y guardar cada split
datasets_info = {}

for split_name, ddf_split in [('train', train_ddf), ('validation', val_ddf), ('test', test_ddf)]:
    
    print(f"\n💾 Procesando {split_name}...")
    
    # Normalizar
    with ProgressBar():
        ddf_normalized = ddf_split.map_partitions(
            lambda x: normalize_partition(x, feature_columns)
        )
    
    # Seleccionar columnas finales
    final_columns = feature_columns + [target_column, 'Time']
    ddf_final = ddf_normalized[final_columns]
    
    # Guardar
    output_path = f"../data/processed/{split_name}_clean.parquet"
    
    try:
        start_time = time.time()
        
        with ProgressBar():
            ddf_final.to_parquet(output_path, write_index=False, compression='snappy')
        
        save_time = time.time() - start_time
        
        # Verificar guardado
        if Path(output_path).exists():
            total_size = sum(f.stat().st_size for f in Path(output_path).rglob('*.parquet'))
            size_mb = total_size / (1024**2)
            
            print(f"✅ {split_name}: {size_mb:.1f} MB ({save_time:.1f}s)")
            
            datasets_info[split_name] = {
                'path': output_path,
                'size_mb': size_mb,
                'save_time': save_time
            }
        else:
            print(f"❌ {split_name}: Error - archivo no encontrado")
            
    except Exception as e:
        print(f"❌ {split_name}: Error - {str(e)[:100]}...")

# Resumen
total_size = sum(info.get('size_mb', 0) for info in datasets_info.values())
total_time = sum(info.get('save_time', 0) for info in datasets_info.values())

print(f"\n📊 RESUMEN GUARDADO:")
print(f"• Total datasets: {len(datasets_info)}")
print(f"• Tamaño total: {total_size:.1f} MB")
print(f"• Tiempo total: {total_time:.1f}s")

monitor_memory_usage("Datasets guardados")

💾 GUARDANDO DATASETS...

💾 Procesando train...
[########################################] | 100% Completed | 2.26 ss
✅ train: 56.2 MB (2.3s)

💾 Procesando validation...
[########################################] | 100% Completed | 1.95 ss
✅ validation: 19.4 MB (2.0s)

💾 Procesando test...
[########################################] | 100% Completed | 2.18 ss
✅ test: 17.2 MB (2.2s)

📊 RESUMEN GUARDADO:
• Total datasets: 3
• Tamaño total: 92.7 MB
• Tiempo total: 6.4s
📊 Memoria - Datasets guardados: 487.53 MB


## 7. Serialización del Pipeline

In [9]:
# Serializar componentes del pipeline
print("📦 SERIALIZANDO PIPELINE...")

base_path = '../data/processed'

# 1. Guardar scaler
scaler_path = f"{base_path}/scaler_clean.pkl"
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"✅ Scaler guardado: {scaler_path}")

# 2. Guardar metadatos
pipeline_metadata = {
    'creation_date': datetime.now().isoformat(),
    'analyst': 'Ing. Daniel Varela Perez',
    'contact': 'bedaniele0@gmail.com',
    'phone': '+52 55 4189 3428',
    'version': 'clean',
    
    'feature_info': {
        'total_features': len(feature_columns),
        'feature_columns': feature_columns,
        'target_column': target_column,
        'scaler_type': 'RobustScaler'
    },
    
    'split_info': {
        'train_size': train_size,
        'val_size': val_size,
        'test_size': test_size,
        'train_fraud': int(train_fraud),
        'val_fraud': int(val_fraud),
        'test_fraud': int(test_fraud)
    },
    
    'datasets_info': datasets_info
}

metadata_path = f"{base_path}/pipeline_metadata_clean.json"
with open(metadata_path, 'w') as f:
    json.dump(pipeline_metadata, f, indent=2)
print(f"✅ Metadatos guardados: {metadata_path}")

# 3. Script de carga
loader_script = f'''#!/usr/bin/env python3
"""
Script para cargar pipeline ETL limpio
Generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""
import pickle
import json
import dask.dataframe as dd

def load_clean_datasets():
    """Carga datasets limpios"""
    train = dd.read_parquet('{base_path}/train_clean.parquet')
    val = dd.read_parquet('{base_path}/validation_clean.parquet')
    test = dd.read_parquet('{base_path}/test_clean.parquet')
    return train, val, test

def load_scaler():
    """Carga scaler"""
    with open('{base_path}/scaler_clean.pkl', 'rb') as f:
        return pickle.load(f)

def load_metadata():
    """Carga metadatos"""
    with open('{base_path}/pipeline_metadata_clean.json', 'r') as f:
        return json.load(f)

if __name__ == "__main__":
    print("Cargando pipeline ETL limpio...")
    train, val, test = load_clean_datasets()
    scaler = load_scaler()
    metadata = load_metadata()
    
    print(f"Train: {{len(train):,}} filas")
    print(f"Val: {{len(val):,}} filas")
    print(f"Test: {{len(test):,}} filas")
    print(f"Features: {{len(metadata['feature_info']['feature_columns'])}}")
    print(f"Scaler: {{type(scaler).__name__}}")
'''

loader_path = f"{base_path}/load_clean_pipeline.py"
with open(loader_path, 'w') as f:
    f.write(loader_script)
print(f"✅ Script de carga: {loader_path}")

print("\n📦 PIPELINE SERIALIZADO:")
print(f"• Scaler: scaler_clean.pkl")
print(f"• Metadatos: pipeline_metadata_clean.json")
print(f"• Loader: load_clean_pipeline.py")

monitor_memory_usage("Pipeline serializado")

📦 SERIALIZANDO PIPELINE...
✅ Scaler guardado: ../data/processed/scaler_clean.pkl
✅ Metadatos guardados: ../data/processed/pipeline_metadata_clean.json
✅ Script de carga: ../data/processed/load_clean_pipeline.py

📦 PIPELINE SERIALIZADO:
• Scaler: scaler_clean.pkl
• Metadatos: pipeline_metadata_clean.json
• Loader: load_clean_pipeline.py
📊 Memoria - Pipeline serializado: 487.54 MB


## 8. Resumen Final

In [10]:
# Resumen final del pipeline limpio
print("🎯 RESUMEN PIPELINE ETL LIMPIO")
print("=" * 50)

print(f"📊 DATOS PROCESADOS:")
print(f"• Train: {train_size:,} filas ({train_fraud} fraudes)")
print(f"• Validation: {val_size:,} filas ({val_fraud} fraudes)")
print(f"• Test: {test_size:,} filas ({test_fraud} fraudes)")

print(f"\n🔧 FEATURES CREADOS:")
print(f"• Total features: {len(feature_columns)}")
print(f"• Nuevas features: {len(new_features)}")
print(f"• Normalización: RobustScaler")

print(f"\n💾 ARCHIVOS GENERADOS:")
for name, info in datasets_info.items():
    print(f"• {name}_clean.parquet ({info['size_mb']:.1f} MB)")

print(f"\n✅ CARACTERÍSTICAS:")
print(f"• ✅ Sin data leakage (split temporal)")
print(f"• ✅ Features normalizadas")
print(f"• ✅ Pipeline serializado")
print(f"• ✅ Demo SMOTE incluido")
print(f"• ✅ Código limpio y funcional")

print(f"\n🚀 LISTO PARA FASE 3: MODELADO ML")
print(f"👨‍💻 Desarrollado por: Ing. Daniel Varela Perez")
print(f"📧 bedaniele0@gmail.com | 📱 +52 55 4189 3428")
print("=" * 50)

monitor_memory_usage("Pipeline completado")

🎯 RESUMEN PIPELINE ETL LIMPIO
📊 DATOS PROCESADOS:
• Train: 172,090 filas (360.0 fraudes)
• Validation: 59,587 filas (61.0 fraudes)
• Test: 53,130 filas (71.0 fraudes)

🔧 FEATURES CREADOS:
• Total features: 39
• Nuevas features: 10
• Normalización: RobustScaler

💾 ARCHIVOS GENERADOS:
• train_clean.parquet (56.2 MB)
• validation_clean.parquet (19.4 MB)
• test_clean.parquet (17.2 MB)

✅ CARACTERÍSTICAS:
• ✅ Sin data leakage (split temporal)
• ✅ Features normalizadas
• ✅ Pipeline serializado
• ✅ Demo SMOTE incluido
• ✅ Código limpio y funcional

🚀 LISTO PARA FASE 3: MODELADO ML
👨‍💻 Desarrollado por: Ing. Daniel Varela Perez
📧 bedaniele0@gmail.com | 📱 +52 55 4189 3428
📊 Memoria - Pipeline completado: 487.54 MB
